# **Import**

In [64]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [65]:
import pandas as pd
import numpy as np
import io
import sklearn 
from datetime import datetime, timedelta

# **Data import AND revise**

In [66]:
i_rate = pd.read_excel('/content/drive/MyDrive/DATA/1. 기준금리 1999~.xlsx')
lead_indicators = pd.read_excel('/content/drive/MyDrive/DATA/2. 경기선행지표.xlsx')
dol_mat = pd.read_excel('/content/drive/MyDrive/DATA/3. 원자제:달러:통화량.xlsx')

#형태 확인
i_rate.shape , lead_indicators.shape , dol_mat.shape

((8190, 2), (7097, 8), (5539, 8))

  
*   총 8190개의 미국 기준금리 데이터 , 7097개의 경기 선행지표 데이터 , 달러화 기준 데이터 및 원자재 데이터를 불러왔습니다. 
*   시계열 기준으로 달러화 및 원자재는 장이 열릴때 수집 되었고 , 금리는 매일 수집되어서 차이가 있습니다. 결측치를 보간하면서 합쳐주도록 하겠습니다.
* 기준은 기준금리로 하겠습니다. = target




In [67]:
print(i_rate.head())
print(i_rate.info())

        DATE  DFEDTAR
0 1999-01-01     4.75
1 1999-01-02     4.75
2 1999-01-03     4.75
3 1999-01-04     4.75
4 1999-01-05     4.75
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8190 entries, 0 to 8189
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   DATE     8190 non-null   datetime64[ns]
 1   DFEDTAR  8190 non-null   float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 128.1 KB
None


In [68]:
print(lead_indicators.head())
print(lead_indicators.info())

        DATE  NASDAQCOM  BAMLH0A0HYM2  HSN1F  UMCSENT  UNRATE      M2      ICSA
0 1999-01-01        NaN           NaN    875    103.9     4.3     NaN       NaN
1 1999-01-02        NaN           NaN    875    103.9     4.3     NaN  331000.0
2 1999-01-04    2208.05          5.63    875    103.9     4.3  4395.0  331000.0
3 1999-01-05    2251.27          5.57    875    103.9     4.3  4395.0  331000.0
4 1999-01-06    2320.86          5.56    875    103.9     4.3  4395.0  331000.0
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7097 entries, 0 to 7096
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   DATE          7097 non-null   datetime64[ns]
 1   NASDAQCOM     7095 non-null   float64       
 2   BAMLH0A0HYM2  7095 non-null   float64       
 3   HSN1F         7097 non-null   int64         
 4   UMCSENT       7097 non-null   float64       
 5   UNRATE        7097 non-null   float64       
 6   M2       

In [69]:
print(dol_mat.head())
print(dol_mat.info())

        Date     NG  ZG     ZI     HG  USD/KRW  USD/EUR  USD/CNY
0 1999-01-04  2.071 NaN  4.902  0.644  1186.25   0.8455   8.2793
1 1999-01-05  1.975 NaN    NaN  0.639  1164.50   0.8501   8.2795
2 1999-01-06  1.931 NaN    NaN  0.645  1157.50   0.8608   8.2795
3 1999-01-07  1.836 NaN  5.204  0.645  1149.50   0.8541   8.2798
4 1999-01-08  1.830 NaN  5.254  0.653  1172.50   0.8639   8.2796
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5539 entries, 0 to 5538
Data columns (total 8 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   Date     5539 non-null   datetime64[ns]
 1   NG       5352 non-null   float64       
 2   ZG       4119 non-null   float64       
 3   ZI       3801 non-null   float64       
 4   HG       5286 non-null   float64       
 5   USD/KRW  5539 non-null   float64       
 6   USD/EUR  5539 non-null   float64       
 7   USD/CNY  5533 non-null   float64       
dtypes: datetime64[ns](1), float64(7)
memory usage: 34

In [70]:
#dol_mat의 열이름 변경 
dol_mat = dol_mat.rename(columns={'Date':'DATE'})
dol_mat

,DATE,NG,ZG,ZI,HG,USD/KRW,USD/EUR,USD/CNY
0,1999-01-04,2.071,NaN,4.902,0.6440,1186.25,0.8455,8.2793
1,1999-01-05,1.975,NaN,NaN,0.6390,1164.50,0.8501,8.2795
2,1999-01-06,1.931,NaN,NaN,0.6450,1157.50,0.8608,8.2795
3,1999-01-07,1.836,NaN,5.204,0.6450,1149.50,0.8541,8.2798
4,1999-01-08,1.830,NaN,5.254,0.6530,1172.50,0.8639,8.2796
...,...,...,...,...,...,...,...,...
5534,2021-05-31,NaN,NaN,NaN,NaN,1106.58,0.8177,6.3688
5535,2021-06-01,3.104,1905.0,28.102,4.6535,1108.38,0.8187,6.3796
5536,2021-06-02,3.075,1909.9,28.204,4.5915,1110.92,0.8188,6.3805
5537,2021-06-03,3.041,1873.3,27.477,4.4630,1116.46,0.8245,6.4036


In [71]:
#DATE 인덱스화 
dol_mat = dol_mat.set_index("DATE")
dol_mat.head() , dol_mat.shape

(               NG  ZG     ZI     HG  USD/KRW  USD/EUR  USD/CNY
 DATE                                                          
 1999-01-04  2.071 NaN  4.902  0.644  1186.25   0.8455   8.2793
 1999-01-05  1.975 NaN    NaN  0.639  1164.50   0.8501   8.2795
 1999-01-06  1.931 NaN    NaN  0.645  1157.50   0.8608   8.2795
 1999-01-07  1.836 NaN  5.204  0.645  1149.50   0.8541   8.2798
 1999-01-08  1.830 NaN  5.254  0.653  1172.50   0.8639   8.2796, (5539, 7))

#**데이터 합치기!**

In [73]:
df_merge_ = pd.merge(i_rate, lead_indicators, on="DATE", how="outer") #irate 8000개 기준 합치기 
df_merge_.head(), df_merge_.shape

(        DATE  DFEDTAR  NASDAQCOM  ...  UNRATE      M2      ICSA
 0 1999-01-01     4.75        NaN  ...     4.3     NaN       NaN
 1 1999-01-02     4.75        NaN  ...     4.3     NaN  331000.0
 2 1999-01-03     4.75        NaN  ...     NaN     NaN       NaN
 3 1999-01-04     4.75    2208.05  ...     4.3  4395.0  331000.0
 4 1999-01-05     4.75    2251.27  ...     4.3  4395.0  331000.0
 
 [5 rows x 9 columns], (8190, 9))

성공적으로 변경된것 같다 이제 나머지 데이터 프레임도 합쳐준다

In [78]:
df_ = pd.merge(df_merge_, dol_mat, on="DATE", how="outer")
df_.head(), df_.shape

(        DATE  DFEDTAR  NASDAQCOM  ...  USD/KRW  USD/EUR  USD/CNY
 0 1999-01-01     4.75        NaN  ...      NaN      NaN      NaN
 1 1999-01-02     4.75        NaN  ...      NaN      NaN      NaN
 2 1999-01-03     4.75        NaN  ...      NaN      NaN      NaN
 3 1999-01-04     4.75    2208.05  ...  1186.25   0.8455   8.2793
 4 1999-01-05     4.75    2251.27  ...  1164.50   0.8501   8.2795
 
 [5 rows x 16 columns], (8191, 16))

성공적으로 합쳐졌으니 결측치를 보간해보자 

In [79]:
df_.isnull().sum()

DATE               0
DFEDTAR            1
NASDAQCOM       1096
BAMLH0A0HYM2    1096
HSN1F           1094
UMCSENT         1094
UNRATE          1094
M2              1096
ICSA            1095
NG              2839
ZG              4072
ZI              4390
HG              2905
USD/KRW         2652
USD/EUR         2652
USD/CNY         2658
dtype: int64

시계열 데이터니까 지우는것 대신 앞의 값으로 채워주도록 한다. 

In [81]:
df_ = df_.fillna(method='ffill')
df_.head()

,DATE,DFEDTAR,NASDAQCOM,BAMLH0A0HYM2,HSN1F,UMCSENT,UNRATE,M2,ICSA,NG,ZG,ZI,HG,USD/KRW,USD/EUR,USD/CNY
0,1999-01-01,4.75,NaN,NaN,875.0,103.9,4.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1999-01-02,4.75,NaN,NaN,875.0,103.9,4.3,NaN,331000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1999-01-03,4.75,NaN,NaN,875.0,103.9,4.3,NaN,331000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1999-01-04,4.75,2208.05,5.63,875.0,103.9,4.3,4395.0,331000.0,2.071,NaN,4.902,0.644,1186.25,0.8455,8.2793
4,1999-01-05,4.75,2251.27,5.57,875.0,103.9,4.3,4395.0,331000.0,1.975,NaN,4.902,0.639,1164.50,0.8501,8.2795


In [82]:
#앞 데이터가 없는 경우 결측치가 그대로 남아있다. 
df_ = df_.fillna(method='bfill') # 뒷방향에서도 채워주기 
df_.head()

,DATE,DFEDTAR,NASDAQCOM,BAMLH0A0HYM2,HSN1F,UMCSENT,UNRATE,M2,ICSA,NG,ZG,ZI,HG,USD/KRW,USD/EUR,USD/CNY
0,1999-01-01,4.75,2208.05,5.63,875.0,103.9,4.3,4395.0,331000.0,2.071,286.4,4.902,0.644,1186.25,0.8455,8.2793
1,1999-01-02,4.75,2208.05,5.63,875.0,103.9,4.3,4395.0,331000.0,2.071,286.4,4.902,0.644,1186.25,0.8455,8.2793
2,1999-01-03,4.75,2208.05,5.63,875.0,103.9,4.3,4395.0,331000.0,2.071,286.4,4.902,0.644,1186.25,0.8455,8.2793
3,1999-01-04,4.75,2208.05,5.63,875.0,103.9,4.3,4395.0,331000.0,2.071,286.4,4.902,0.644,1186.25,0.8455,8.2793
4,1999-01-05,4.75,2251.27,5.57,875.0,103.9,4.3,4395.0,331000.0,1.975,286.4,4.902,0.639,1164.50,0.8501,8.2795


In [83]:
df_.isnull().sum()

DATE            0
DFEDTAR         0
NASDAQCOM       0
BAMLH0A0HYM2    0
HSN1F           0
UMCSENT         0
UNRATE          0
M2              0
ICSA            0
NG              0
ZG              0
ZI              0
HG              0
USD/KRW         0
USD/EUR         0
USD/CNY         0
dtype: int64

**결측치가 완전히 사라진 것을 알 수 있다.**

In [87]:
df_.to_csv('DFEDTAR_data.csv')